In [ ]:
import tensorflow as tf
tf.test.gpu_device_name()

In [ ]:
import math
import numpy as np
import cv2 as cv
import random as rd
import os
import time

def image_none(img_path):
    image=cv.imread(img_path)
    return image

def octile_distance(node1,node2):
    dx= node2[0] - node1[0]
    dy= node2[1] - node1[1]
    f = math.sqrt(2) - 1
    if dx < dy:
        return f * dx + dy
    else:
        return f * dy + dx

def coloring_pixel(image,color,pixel):
    if image is  not None:
        image[pixel[0],pixel[1]]=color
        return image

def define_start_goal(image,start,goal):
    start_color = [255, 0, 102] #pink 
    goal_color =  [0, 0, 204] #blue
    if image is not None:
        # should not be obstacle 
        if np.array_equal(image[start[0],start[1]],[229, 229, 229]) and np.array_equal(image[goal[0],goal[1]],[229, 229, 229]):
            image = coloring_pixel(image,start_color,start)
            image = coloring_pixel(image,goal_color,goal) 
            return image
        else: 
            print("Goal or Start is an obstacle, Redefine it!")
            return image

def draw_path(img_path,path):    
    image=cv.imread(img_path)
    if image is not None:
        for tuple in path:
            print(tuple)
            image[tuple[1],tuple[0]]=[0, 0, 255] #red
    return image


def isValid(img_path,node):
    image=cv.imread(img_path,0)
    if image is not None:
        return  image[node[0],node[1]] == 229 



def imageToMatrix(img_path):
    image=cv.imread(img_path,0)
    rows,cols=(1073,1073)    
    # image (grey scale) to a matrix of 0 and 1 
    for i in range(rows):
        for j in range(cols):
            if image[i,j] == 0 or image[i,j] == 99: 
                image[i,j] = 0 #obstacle
            elif image[i,j] == 229:
                image[i,j] = 1
    return image                
    


In [ ]:

""" output structure: (room_name,method used, type de test, path, path length, operations, time)"""
class room_path:
    def __init__(self,room_name,method,test,path,path_length,operations,time):
        self.room_name=room_name
        self.method=method
        self.test=test
        self.path=path
        self.path_length=path_length
        self.operations=operations
        self.time=time
    def path_info(self):
        path_info = "room name : " + str(self.room_name) + ", path length : " + str(self.path_length) + ", number of operations : " + str(self.operations) + ", time : " + str(self.time)
        return path_info
        

def findNeighbors(matrix,node):
    #check validity and return neighbors to choose randomly one of them
    neighbors=[]

    if matrix[node[0]+1,node[1]] == 1:
        neighbors.append((node[0]+1,node[1]))

    if matrix[node[0],node[1]+1]==1:
        neighbors.append((node[0],node[1]+1))

    if matrix[node[0]+1,node[1]+1]==1:
        neighbors.append((node[0]+1,node[1]+1))

    if matrix[node[0]-1,node[1]]==1:
        neighbors.append((node[0]-1,node[1]))
    
    if matrix[node[0],node[1]-1]==1:
        neighbors.append( (node[0],node[1]-1))

    if matrix[node[0]-1,node[1]-1]==1:
        neighbors.append((node[0]-1,node[1]-1))

    if matrix[node[0]-1,node[1]+1]==1:
        neighbors.append((node[0]-1,node[1]+1))

    if matrix[node[0]+1,node[1]-1]==1:
        neighbors.append((node[0]+1,node[1]-1)) 

    return neighbors


def generatePath(matrix,start,end): 
    path=[]
    path.append(start)
    node=start 
    while(node!=end):
        neighbors = findNeighbors(matrix,node)
        node=neighbors[rd.randint(0,len(neighbors)-1)]
        path.append(node)
    return path

 
def generateNeighboringPath(matrix,path): 
    x=rd.randint(0,len(path)-2)
    neighbors=findNeighbors(matrix,path[x])
    output=path.copy()
    output[x]=neighbors[rd.randint(0,len(neighbors)-2)] 
    output[x+1:]=generatePath(matrix,output[x+1],output[-1])
    return output

def cost(path):
    distance=0
    for i in range(len(path)-1):
        distance+=octile_distance(path[i],path[i+1])
    return distance

def sa(matrix,start,end , parameters):  
    CT = parameters['IT']  # Current temperature
    N= parameters['N']
    NT= parameters['NT'] 
    alpha = parameters['alpha']
    FT = parameters['FT']
    CP = generatePath(matrix,start,end) # current path (random)
    CL = cost(CP) 
    OP = CP # optimal path
    OL = CL
    for i in range(N):
        print("i= ",i)
        for j in range(NT):
            RP = generateNeighboringPath(matrix,CP)
            RL = cost(RP)  
            if RL < CL:
                CP = RP
                CL = RL 
            else: 
                p = math.exp(-(RL-CL)/CT) # Transition Probability 
                r=rd.randint(0,1)
                if r<p:
                    CP = RP
                    CL = RL 
            if CL < OL:
                OP = CP
                OL = CL
        CT *= alpha 
        if CT <= FT:
            break 
    return OP , OL 

def SA(img_path,start,end,image_name,parameters):
    image=imageToMatrix(img_path)
    op , ol = sa(image,start,end,parameters) 
    #Draw path in the image and save it
    image=cv.imread(img_path)
    new_image = draw_path(img_path,op) 
    saving_path = '/content/'
    cv.imwrite(os.path.join(saving_path ,image_name), new_image)
    
    return op, ol 


In [ ]:


# TEST
folder_dir="/content/"
file=open("/content/all_results.txt","w")

IT = [ 2,4,7]
FT = [ 1,2,3 ]
N =[ 2,4 ] 
NT = [ 2,4] 
alpha = [ 0.9 , 0.3 ]
images=["8room_000.png","16room_000.png","32room_000.png","64room_000.png"]

file.write("\nPARAMETER TUNING \n")

start = (10, 9)
end = (14, 40)
easy_output =[]

i=0
for image in images:
    for it in IT:
        for ft in FT:
            for n in N:
                for nt in NT:
                    for a in alpha:
                    # Test the 4 images and save results    
                       # Initializing parameters
                        parameters = {}
                        parameters['IT'] = it     # Initial temperature
                        parameters['FT'] = ft     # Final temperature 
                        parameters['N']  = n      # Number of iterations
                        parameters['NT']= nt       # Number of iterations per Temperature 
                        parameters['alpha'] = a  # Geometric Coefficient alpha
                        img_path=folder_dir+"/"+str(image)
                        img=cv.imread(img_path,0)
                        if img is not None:
                            if isValid(img_path,start):
                                if isValid(img_path,end):
                                    print(str(image))
                                    start_time = time.time()
                                    path , length = SA(img_path,start,end,str(i)+"new_"+str(image),parameters)
                                    print("--- %s seconds ---" % (time.time() - start_time))
                                    easy_output.append(room_path.room_path(str(i)+"new_"+str(image),"SA","easy" ,path, length,0,time.time() - start_time))
                                    pathInfo = easy_output[-1].path_info()
                                    print(pathInfo)
                                    file.write(pathInfo +"it: "+str(it)+" ft: "+str(ft)+" n: "+str(n)+" nt: "+str(nt)+" alpha: "+str(alpha) + "\n")
                                    i=i+1
                                else:
                                    print(str(image) ,"invalid end")
                            else:
                                print(str(image),"invalid start")
                        else:
                            print("image is None")

